## Tool-Column Transformer - Encoding

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer ## to fill the missing values
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.head()

## age,fever,mild --> ordinal encoding
## gender,city --->one hot encoder

age  gender  fever cough     city has_covid
0   60    Male  103.0  Mild  Kolkata        No
1   27    Male  100.0  Mild    Delhi       Yes
2   42    Male  101.0  Mild    Delhi        No
3   31  Female   98.0  Mild  Kolkata        No
4   65  Female  101.0  Mild   Mumbai        No

In [5]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [7]:
X_train

age  gender  fever   cough       city
43   22  Female   99.0    Mild  Bangalore
90   59  Female   99.0  Strong      Delhi
35   82  Female  102.0  Strong  Bangalore
5    84  Female    NaN    Mild  Bangalore
61   81  Female   98.0  Strong     Mumbai
..  ...     ...    ...     ...        ...
79   48  Female  103.0    Mild    Kolkata
37   55    Male  100.0    Mild    Kolkata
23   80  Female   98.0    Mild      Delhi
2    42    Male  101.0    Mild      Delhi
32   34  Female  101.0  Strong      Delhi

[80 rows x 5 columns]

## 1. Without column transformer - encoding


In [16]:
## missing values--> replce with mean values

# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [11]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [13]:
from sklearn.preprocessing import OneHotEncoder

# Initialize OneHotEncoder with correct parameter
ohe = OneHotEncoder(drop='first', sparse_output=False)

# Fit and transform the training data
X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])

# Transform the test data (use transform, not fit_transform)
X_test_gender_city = ohe.transform(X_test[['gender', 'city']])

# Print shape
print(X_train_gender_city.shape)
print(X_test_gender_city.shape)


(80, 4)
(20, 4)


In [14]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [15]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

### 2. With Column Transformers - Encoding

In [17]:
from sklearn.compose import ColumnTransformer

In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

# Define the ColumnTransformer with the correct parameter
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),  # Handling missing values
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),  # Ordinal encoding
    ('tnf3', OneHotEncoder(sparse_output=False, drop='first'), ['gender', 'city'])  # One-hot encoding with first category drop
], remainder='passthrough')  # Keep other columns as is


In [20]:
transformer.fit_transform(X_train).shape

(80, 7)

In [21]:
transformer.transform(X_test).shape

(20, 7)